In [1]:
import os
import glob
import socket
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from numba import float32, float64, guvectorize

In [2]:
hostname = socket.gethostname()
print(f"Detected host: {hostname}")
if hostname == 'UU083283':
    data_dir = '../../../data/data_iHESP_h3/*'    
else:
    data_dir = None

Detected host: UU083283


In [3]:
files = sorted([os.path.realpath(p) for p in glob.glob(data_dir)])
files

['/Users/jasperdejong/Documents/PhD/Geo/data/data_iHESP_h3/B.E.13.B1950TRC5.ne30g16.ihesp24.sai2020.01.cam.h0.2020-01.nc',
 '/Users/jasperdejong/Documents/PhD/Geo/data/data_iHESP_h3/B.E.13.B1950TRC5.ne30g16.ihesp24.sai2020.01.cam.h3.2098-12-13-64800.nc',
 '/Users/jasperdejong/Documents/PhD/Geo/data/data_iHESP_h3/B.E.13.B1950TRC5.ne30g16.ihesp24.sai2020.01.cam.h3.2099-01-01-00000.nc',
 '/Users/jasperdejong/Documents/PhD/Geo/data/data_iHESP_h3/B.E.13.B1950TRC5.ne30g16.ihesp24.sai2020.01.cam.h4.2098-11-25-43200.nc',
 '/Users/jasperdejong/Documents/PhD/Geo/data/data_iHESP_h3/B.E.13.B1950TRC5.ne30g16.ihesp24.sai2020.01.cam.h4.2099-01-01-00000.nc',
 '/Users/jasperdejong/Documents/PhD/Geo/data/data_iHESP_h3/Vshear']

In [65]:
if hostname == 'UU083283':
    ds0 = xr.open_dataset(files[0])
    ds3a = xr.open_dataset(files[1])
    ds3b = xr.open_dataset(files[2])
    ds4a = xr.open_dataset(files[3])
    ds4b = xr.open_dataset(files[4])
    print("files:",[f.split(".")[-2] for f in files])
    print("has PS:",[hasattr(ds,'PS') for ds in [ds0,ds3a,ds3b,ds4a,ds4b]])
    print("[time, hyam, hybm, P0] equal in h3 and h4 files:")
    print("    2098: ",end="")
    print(all(ds3a.time == ds4a.time), end=", ")
    print(all(ds3a.hyam == ds4a.hyam), end=", ")
    print(all(ds3a.hybm == ds4a.hybm), end=", ")
    print((ds3a.P0 == ds4a.P0).data)
    print("    2099: ",end="")
    print(all(ds3b.time == ds4b.time), end=", ")
    print(all(ds3b.hyam == ds4b.hyam), end=", ")
    print(all(ds3b.hybm == ds4b.hybm), end=", ")
    print((ds3b.P0 == ds4b.P0).data)
    ds3a = xr.combine_by_coords([ds3a,ds4a.PS],join='inner')
    ds3b = xr.combine_by_coords([ds3b,ds4b.PS],join='inner')
    print('fixing ...')
    print('PS equal in h3 and h4 files:',
          (ds3a.PS==ds4a.PS).all().data,(ds3b.PS==ds4b.PS).all().data)

files: ['2020-01', '2098-12-13-64800', '2099-01-01-00000', '2098-11-25-43200', '2099-01-01-00000']
has PS: [True, False, False, True, True]
[time, hyam, hybm, P0] equal in h3 and h4 files:
    2098: True, True, True, True
    2099: True, True, True, True
fixing ...
PS equal in h3 and h4 files: True True


In [106]:
def press_from_hybrid(ds: xr.Dataset, loc='mid'):
    """Calculate pressure from hybrid coefficients
    
    Params:
    -------
    ds : xarray.Dataset 
        dataset containing hybrid parameters
    loc : ['mid','int'], str
        determine pressure at hybrid level midpoints or interfaces
    inplace : bool
        if True, 
    
    Returns:
    --------
    xarray.Dataset
        dataset including atmospheric pressure
    """
    loc_opts = dict({'mid':'midpoints','int':'interfaces'})
    assert (loc in loc_opts), f"loc must be in {list(loc_opts)}"
    print(f"calculating pressure on model level {loc_opts[loc]}")
    if loc == 'mid':
        P = ds.hyam * ds.P0 + ds.hybm * ds.PS
    elif loc == 'int':
        P = ds.hyai * ds.P0 + ds.hybi * ds.PS
    P.attrs = {'long_name':'atmospheric pressure','units':ds.PS.units,
               'cell_methods':ds.PS.cell_methods}
    ds['P'] = P
    return ds


@guvectorize(
    "(float32[:], float64[:], float64[:], float32[:])",
    " (n), (n), (m) -> (m)",
    nopython=True,
)
def interp1d_gu(f, x, xi, out):
    """Interpolate field f(x) to xi in ln(x) coordinates."""
    i, imax, x0, f0 = 0, len(xi), x[0], f[0]
    while xi[i]<x0 and i < imax:
        out[i] = np.nan      
        i = i + 1 
    for x1,f1 in zip(x[1:], f[1:]):
        while xi[i] <= x1 and i < imax:
            out[i] = (f1-f0)/np.log(x1/x0)*np.log(xi[i]/x0)+f0
            i = i + 1
        x0, f0 = x1, f1
    while i < imax:
        out[i] = np.nan
        i = i + 1


def wind_shear(ds: xr.Dataset, P1=850, P2=250):
    """Calculate wind shear between two pressure levels
    
    Params:
    -------
    ds : xarray.Dataset
        dataset containing velocity data on model levels
    P1, P2 : int or float
        pressure levels (hPa) to take difference from
    P : xarray.DataArray
        atmospheric pressure at model levels (Pa)
        
    Returns:
    --------
    xarray.DataArray
        absolute difference between wind speeds on P1 and P2
    """
    print(f"calculating wind shear between {P1}-{P2} hPa")
    Pinterp = xr.DataArray([P1,P2],dims='np') * 100
    V = np.sqrt(ds.U**2 + ds.V**2)
    if 'P' not in ds:
        ds['P'] = press_from_hybrid(ds)
    Vinterp = xr.apply_ufunc(
        interp1d_gu, V, ds.P, Pinterp,
        input_core_dims=[['lev'],['lev'],['np']],
        output_core_dims=[['np']],
        exclude_dims=set(['lev']),
        output_dtypes=['float32'])
    Vshear = np.abs(Vinterp.isel(np=1) - Vinterp.isel(np=0))
    Vshear.attrs = {'name':'Vshear','units':'m/s', 'cell_methods':ds.U.cell_methods,
                    'long_name':f'absolute difference in wind speed between {P1} hPa and {P2} hPa'}
    return Vshear
    
    
def main():
    for i,ds in enumerate([ds3a, ds3b]):
        P = press_from_hybrid(ds3a,loc='mid')
        Vshear = wind_shear(ds3a, P=P)
        Vshear.to_netcdf(f"Vshear{i}.nc")
        

In [108]:
#ds3ap = press_from_hybrid(ds3a,loc='mid')
Vshear = wind_shear(ds3a)

calculating wind shear between 850-250 hPa


In [109]:
ds3a

<xarray.Dataset>
Dimensions:        (lev: 30, ilev: 31, cosp_prs: 7, cosp_tau: 7, cosp_scol: 50,
                    cosp_ht: 40, cosp_sr: 15, cosp_sza: 5, time: 73, nbnd: 2,
                    ncol: 48602)
Coordinates:
  * lev            (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev           (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * cosp_prs       (cosp_prs) float64 900.0 740.0 620.0 500.0 375.0 245.0 90.0
  * cosp_tau       (cosp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 219.5
  * cosp_scol      (cosp_scol) int32 1 2 3 4 5 6 7 8 ... 43 44 45 46 47 48 49 50
  * cosp_ht        (cosp_ht) float64 240.0 720.0 1.2e+03 ... 1.848e+04 1.896e+04
  * cosp_sr        (cosp_sr) float64 0.605 2.1 4.0 6.0 ... 70.0 539.5 1.004e+03
  * cosp_sza       (cosp_sza) float64 0.0 15.0 30.0 45.0 60.0
  * time           (time) object 2098-12-13 18:00:00 ... 2098-12-31 18:00:00
Dimensions without coordinates: nbnd, ncol
Data variables: (12/43)
    hyam           (lev) float64 ...
    hybm           (lev) float64 ...
    P0             float64 ...
    hyai           (ilev) float64 ...
    hybi           (ilev) float64 ...
    cosp_prs_bnds  (cosp_prs, nbnd) float64 ...
    ...             ...
    T              (time, lev, ncol) float32 ...
    U              (time, lev, ncol) float32 -26.53 -26.12 ... 6.224 5.342
    V              (time, lev, ncol) float32 -1.463 -1.471 ... 0.7768 1.452
    Z3             (time, lev, ncol) float32 ...
    PS             (time, ncol) float32 ...
    P              (lev, time, ncol) float64 364.3 364.3 ... 9.901e+04 1.003e+05
Attributes:
    np:               4
    ne:               30
    Conventions:      CF-1.0
    source:           CAM
    case:             B.E.13.B1950TRC5.ne30g16.ihesp24.sai2020.01
    title:            UNSET
    logname:          renew1
    host:             
    Version:          $Name$
    revision_Id:      $Id$
    initial_file:     b.e13.B1950TRC5.ne30_g16.ihesp24_1950-2050.002.cam.i.20...
    topography_file:  /projects/0/nwo2021025/inputdata/atm/cam/topo/USGS-gtop...

In [18]:
P = press_from_hybrid(ds3a,ds4a.PS,loc='int')

In [36]:
Vshear

<xarray.DataArray (time: 73, ncol: 48602)>
array([[-56.214787 , -60.515728 , -51.997612 , ...,  -5.797479 ,
          5.367695 ,   9.396834 ],
       [-31.68726  , -42.521793 , -44.625336 , ...,   7.234621 ,
         10.500195 ,  11.184942 ],
       [-97.43198  , -50.567474 , -41.11863  , ...,  11.7417755,
         26.867958 ,  30.006672 ],
       ...,
       [-50.932266 , -46.38887  , -20.332495 , ...,  35.7412   ,
         15.237794 ,  15.1998825],
       [-86.441574 , -76.310165 , -41.270496 , ...,  37.510365 ,
         43.29175  ,  53.336845 ],
       [-78.002365 , -87.83097  , -64.157616 , ...,  -7.8124404,
         57.60205  ,  42.882065 ]], dtype=float32)
Coordinates:
  * time     (time) object 2098-12-13 18:00:00 ... 2098-12-31 18:00:00
Dimensions without coordinates: ncol